## CNN sentence classifier - Hyperparameter tuning with reduced sentence length


In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 50


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 14.6 ms, sys: 7.72 ms, total: 22.3 ms
Wall time: 21.3 ms


## Tokenize the training data set and build a vocabulary from it


In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31.9 s, sys: 72.9 ms, total: 31.9 s
Wall time: 31.6 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [14]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen50.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16], [16,32], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96.96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal50",
                      model_dir        = "../models/CNNBal50",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.6775 - accuracy: 0.5658 - val_loss: 0.5998 - val_accuracy: 0.6939
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-182920/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4842 - accuracy: 0.7667 - val_loss: 0.3461 - val_accuracy: 0.8721
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-182920/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2981 - accuracy: 0.8786 - val_loss: 0.3594 - val_accuracy: 0.8626
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1866 - accuracy: 0.9322 - val_loss: 0.3482 - val_accuracy: 0.8761
Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1145 - accuracy: 0.9600 - val_loss: 0.2792 - val_accuracy: 0.9046
INFO:tensorflow:Assets written to

288/288 [==============================] - 1s 5ms/step - loss: 0.0318 - accuracy: 0.9890 - val_loss: 0.2899 - val_accuracy: 0.9263
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0348 - accuracy: 0.9897 - val_loss: 0.4517 - val_accuracy: 0.9051
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0274 - accuracy: 0.9911 - val_loss: 0.3318 - val_accuracy: 0.9304
Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0209 - accuracy: 0.9923 - val_loss: 0.3279 - val_accuracy: 0.9313
Epoch 12/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0217 - accuracy: 0.9935 - val_loss: 0.3691 - val_accuracy: 0.9277
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0238 - accuracy: 0.9931 - val_loss: 0.3567 - val_accuracy: 0.9308
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0205 - accuracy: 0.9940 - val_loss: 0.2158 - val_accuracy: 0.9571
Epoch 15/20


288/288 [==============================] - 2s 5ms/step - loss: 0.0102 - accuracy: 0.9973 - val_loss: 0.2055 - val_accuracy: 0.9629
Epoch 19/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0145 - accuracy: 0.9944 - val_loss: 0.3663 - val_accuracy: 0.9426
Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0054 - accuracy: 0.9985 - val_loss: 0.2678 - val_accuracy: 0.9575



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.7136 - accuracy: 0.6229 - val_loss: 0.5789 - val_accuracy: 0.7048
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-183207/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4347 - accuracy: 0.8016 - val_loss: 0.3481 - val_accuracy: 0.8490
INFO:tensorflow:Assets written to: .

288/288 [==============================] - 1s 5ms/step - loss: 0.1242 - accuracy: 0.9562 - val_loss: 0.2598 - val_accuracy: 0.9028
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-183314/assets
Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0726 - accuracy: 0.9755 - val_loss: 0.1506 - val_accuracy: 0.9525
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-183314/assets
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0525 - accuracy: 0.9819 - val_loss: 0.2687 - val_accuracy: 0.9322
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0434 - accuracy: 0.9855 - val_loss: 0.4364 - val_accuracy: 0.9019
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0413 - accuracy: 0.9852 - val_loss: 0.3094 - val_accuracy: 0.9218
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0257 - accuracy: 0.9921 - val_loss: 0.1697 - val_accuracy: 0.9598
Epoch 10/

288/288 [==============================] - 1s 5ms/step - loss: 0.0181 - accuracy: 0.9944 - val_loss: 0.2937 - val_accuracy: 0.9485
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0174 - accuracy: 0.9945 - val_loss: 0.1553 - val_accuracy: 0.9684
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-183421/assets
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0152 - accuracy: 0.9953 - val_loss: 0.3265 - val_accuracy: 0.9421
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0097 - accuracy: 0.9970 - val_loss: 0.6689 - val_accuracy: 0.8924
Epoch 16/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0093 - accuracy: 0.9974 - val_loss: 0.4171 - val_accuracy: 0.9322
Epoch 17/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.5912 - val_accuracy: 0.9051
Epoch 18/20
288/288 [==============================] - 1s 5ms/step - loss: 0.01

...

Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0579 - accuracy: 0.9825 - val_loss: 0.2499 - val_accuracy: 0.9155
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-193002/assets
Epoch 6/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0311 - accuracy: 0.9945 - val_loss: 0.3636 - val_accuracy: 0.8956
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0243 - accuracy: 0.9937 - val_loss: 0.4406 - val_accuracy: 0.8811
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0184 - accuracy: 0.9953 - val_loss: 0.2312 - val_accuracy: 0.9322
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-193002/assets
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0105 - accuracy: 0.9981 - val_loss: 0.1622 - val_accuracy: 0.9566
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-193002/assets
Epoch 10/20
288/288 [==============================] - 2s 6ms/ste

288/288 [==============================] - 2s 6ms/step - loss: 0.0125 - accuracy: 0.9954 - val_loss: 0.4328 - val_accuracy: 0.9204
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0113 - accuracy: 0.9968 - val_loss: 0.3044 - val_accuracy: 0.9363
Epoch 14/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0098 - accuracy: 0.9966 - val_loss: 0.3201 - val_accuracy: 0.9381
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.3848 - val_accuracy: 0.9322
Epoch 16/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0081 - accuracy: 0.9973 - val_loss: 0.4861 - val_accuracy: 0.9159
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0080 - accuracy: 0.9978 - val_loss: 0.2512 - val_accuracy: 0.9494
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0124 - accuracy: 0.9968 - val_loss: 0.3811 - val_accuracy: 0.9344
Epoch 19/20

288/288 [==============================] - 2s 7ms/step - loss: 0.0040 - accuracy: 0.9987 - val_loss: 0.6888 - val_accuracy: 0.9218



##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 128
Adding Convolution: Kernel Size: 8, Filter Count: 128
Adding Convolution: Kernel Size: 16, Filter Count: 128
Adding Convolution: Kernel Size: 32, Filter Count: 128
Epoch 1/20
288/288 [==============================] - 3s 8ms/step - loss: 0.5899 - accuracy: 0.6750 - val_loss: 0.4785 - val_accuracy: 0.7699
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-193333/assets
Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.2824 - accuracy: 0.8839 - val_loss: 0.2475 - val_accuracy: 0.8996
INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-193333/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1196 - accuracy: 0.9634 - val_loss: 0.1733 - val_accuracy: 0

INFO:tensorflow:Assets written to: ../models/CNNBal50/210408-193457/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0285 - accuracy: 0.9920 - val_loss: 0.2574 - val_accuracy: 0.9322
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0177 - accuracy: 0.9947 - val_loss: 0.3863 - val_accuracy: 0.9082
Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0260 - accuracy: 0.9917 - val_loss: 0.2314 - val_accuracy: 0.9462
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0099 - accuracy: 0.9970 - val_loss: 0.4225 - val_accuracy: 0.9204
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0073 - accuracy: 0.9974 - val_loss: 0.3543 - val_accuracy: 0.9127
Epoch 10/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0208 - accuracy: 0.9930 - val_loss: 0.4103 - val_accuracy: 0.9141
Epoch 11/20
288/288 [==============================] - 2s 7ms/step - los

288/288 [==============================] - 2s 6ms/step - loss: 0.0114 - accuracy: 0.9962 - val_loss: 0.2347 - val_accuracy: 0.9503
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0087 - accuracy: 0.9972 - val_loss: 0.4397 - val_accuracy: 0.9105
Epoch 14/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0089 - accuracy: 0.9975 - val_loss: 0.5084 - val_accuracy: 0.9055
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0101 - accuracy: 0.9966 - val_loss: 0.2978 - val_accuracy: 0.9462
Epoch 16/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.4575 - val_accuracy: 0.9204
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0056 - accuracy: 0.9984 - val_loss: 0.3525 - val_accuracy: 0.9408
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0079 - accuracy: 0.9971 - val_loss: 0.5041 - val_accuracy: 0.9241
Epoch 19/20





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/